In [1]:
from database_connect import *
from tqdm import tqdm
from bs4 import BeautifulSoup

In [79]:
connection.close()

In [2]:
connection, cursor = master_db_login()

In [3]:
query = "SELECT brand_id, BrandName, BrandSite, ShopifySite FROM master_table WHERE ShopifySite LIKE '%.%' LIMIT 50;"

In [4]:
df = pd.read_sql_query(query, connection)

C:\Users\PF000107\AppData\Local\Temp\ipykernel_15972\498374113.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, connection)


In [5]:
df.shape

(50, 4)

In [6]:
def get_soup(url):
    try:
        # Send a GET request to the URL
        response = requests.get(url)
        
        # Check if the request was successful (status code 200)
        if response.status_code == 200:
            # Parse the HTML content of the page using BeautifulSoup
            soup = BeautifulSoup(response.text, 'html.parser')
            return soup
        else:
            print('Failed to retrieve the page. Status code:', response.status_code)
            return None
    except Exception as e:
        print('An error occurred:', str(e))
        return None

In [7]:
def extract_instagram_urls(page_source):
    
    # Define the regular expression pattern
    pattern = re.compile(r'https:\\\/\\\/www\.instagram\.com\\\/[\w\d_]+\/?')

    # Search for the pattern in the page source
    matches = pattern.findall(page_source)

    # Filter URLs that contain "facebook.com"
    instagram_urls = [url for url in matches if 'instagram.com' in url]

    return instagram_urls[0]

In [8]:
def extract_instagram_urls_2(page_source):
    
    # Define the regular expression pattern
    pattern = re.compile(r'https?://(?:www\.)?instagram\.com/[\w\d_-]+(?:/\?[^"\'\s]*)?')

    # Search for the pattern in the page source
    matches = pattern.findall(page_source)

    # Filter URLs that contain "facebook.com"
    instagram_urls = [url for url in matches if 'instagram.com' in url]

    return instagram_urls[0]

In [9]:
df['insta_urls'] = None
for row in tqdm(df.itertuples()):
    
# for row in nandu_df.itertuples(): 
# 
    try: 
#         print(row.BrandSiteUpdated)
        response = requests.get(row.BrandSite,timeout=12)
        soup = BeautifulSoup(response.text, 'html.parser')
        social_elements = soup.find_all(class_=lambda x: x and 'social' in x)
        
        try:
            insta_url = extract_instagram_urls(str(social_elements))
        except:
            try:
                insta_url = extract_instagram_urls_2(str(social_elements))
            except:
                try:
                    insta_url = extract_instagram_urls(str(soup))
                except:
                    insta_url = extract_instagram_urls_2(str(soup))

        insta_url = insta_url.replace(r'\/', '/')

        print(f'insta url: {insta_url}')
        df.at[row.Index, 'insta_urls'] = insta_url
        
        # Increment the last_processed_index
        # last_processed_index += 1
        
        # Save the last_processed_index to the text file
        # save_last_processed_index(last_processed_index)
    except:
        pass 

0it [00:00, ?it/s]

1it [00:00,  2.32it/s]ERROR:root:insta url: https://www.instagram.com/509inc
2it [00:01,  1.27it/s]ERROR:root:insta url: https://www.instagram.com/7amenfant
4it [00:03,  1.12it/s]ERROR:root:insta url: https://www.instagram.com/agoodcom
5it [00:04,  1.11it/s]ERROR:root:insta url: https://www.instagram.com/averieofficial
6it [00:05,  1.08it/s]ERROR:root:insta url: https://www.instagram.com/abccarpetandhome
7it [00:06,  1.08s/it]ERROR:root:insta url: https://www.instagram.com/aberlitegrooming
8it [00:07,  1.07it/s]ERROR:root:insta url: https://www.instagram.com/actandacre
9it [00:08,  1.01s/it]ERROR:root:insta url: https://instagram.com/aday
10it [00:09,  1.05it/s]ERROR:root:insta url: https://www.instagram.com/fablepets
11it [00:10,  1.07it/s]ERROR:root:insta url: https://www.instagram.com/alexisrusselljewelry
12it [00:10,  1.19it/s]ERROR:root:insta url: https://www.instagram.com/allbirds
13it [00:11,  1.36it/s]ERROR:root:insta url: https://www.instagram.com/allegianceflagsupply
14it [00

In [10]:
df

,brand_id,BrandName,BrandSite,ShopifySite,insta_urls
0,2,31 bits,http://www.31bits.com/,http://www.31bits.com/,None
1,4,509,http://www.ride509.com/offroad,https://ride509.com/,https://www.instagram.com/509inc
2,5,Weekdays and Weekends,https://weekdaysandweekends.com/,https://weekdaysandweekends.com/,None
3,6,7AM Enfant,http://www.7amenfant.com/,http://www.7amenfant.com/,https://www.instagram.com/7amenfant
4,8,agood company,https://agood.com/,https://agood.com/,https://www.instagram.com/agoodcom
5,17,Averie,http://www.aavaswim.com/,http://www.aavaswim.com/,https://www.instagram.com/averieofficial
6,18,abc carpet & home,http://www.abchome.com/,http://www.abchome.com/,https://www.instagram.com/abccarpetandhome
7,19,Aberlite,http://www.aberlite.com/,http://www.aberlite.com/,https://www.instagram.com/aberlitegrooming
8,23,Act+Acre,http://www.actandacre.com/,http://www.actandacre.com/,https://www.instagram.com/actandacre
9,27,Aday,http://www.thisisADAY.com/,http://www.thisisADAY.com/,https://instagram.com/aday


In [74]:
response = requests.get("http://www.beachriot.com/",timeout=12)
soup = BeautifulSoup(response.text, 'html.parser')

In [75]:
social_elements = soup.find_all(class_=lambda x: x and 'social' in x)

In [76]:
social_elements

[]